# Experiments - Primitive solutions to Quaternion Embedding Problem

We start by implementing algorithm 5.3 to check if a solution is primitive.

In [2]:
from hnf import upper_hnf_matrix, basis_to_matrix

def check_primitive(alpha, O, to_f_matrix=None):
    """
    Given solution alpha in an order O, returns True if solution is primitive, False otherwise.

    Works by computing the matrix mapping alpha's coefficients in 1,i,j,k to coefficients of upper triangular HNF matrix.
        For efficienty, to avoid repeating this computation you can provide it as 'to_f_matrix'.
    """
    # compute coeffs of alpha in upper triangular HNF basis
    if to_f_matrix == None:
        to_f_matrix = upper_hnf_matrix(basis_to_matrix(O.basis())).inverse().transpose()
    f_coeffs = (matrix(alpha.coefficient_tuple()) * to_f_matrix).list()
    # remove duplicate values
    S = [f_coeffs[1]]
    if f_coeffs[2] not in S: S.append(f_coeffs[2])
    if f_coeffs[3] not in S: S.append(f_coeffs[3])
    S = [m for m in S if m != 0]
    if len(S) == 0: return False
    # check gcd
    return gcd(S) == 1

### Example 1

Consider $p=419$ and trying to embed $\mathbb{Z}[\sqrt{-30273}]$ into order $\mathcal{O}_0 = End(E_{j=1728})$. Using the general algorithm we find solution $\alpha = 1/2i + 4j + 15/2k$ of norm $30273$ and trace $0$. Note that since $\mathbb{Z}[\sqrt{-30273}]$ is maximal in $\mathbb{Q}[\sqrt{-30273}]$ it is impossible to extend the embedding to a superorder, hence the solution must be primitive. We can check this:

In [3]:
p = 419
B.<i,j,k> = QuaternionAlgebra(-1, -p)
MO = B.maximal_order()
basis = MO.basis()
alpha = 1/2*i + 4*j + 15/2*k

check_primitive(alpha, MO)

True

## Combining Algorithms 5.1 and 5.3

Now we combine Algorithm 5.1, for finding embeddings, with Algorithm 5.3 to check whether each embedding is primitive. This code stops only when a primitive solution is found or the search covers the whole space.

In [4]:
from algorithm_5_1 import find_element_defining_embedding

def find_element_defining_primitive_embedding(O, d, t, all_slns=False):
    """
    Finds element of order 'O' of trace t and norm d that is a primitive solution.
    """
    to_f_matrix = basis_to_matrix(O.basis()).inverse().transpose()

    def check_sln(alpha, k):
        return check_primitive(alpha, O, to_f_matrix)
    
    # We pass the above function into the search as a validity check, so we know the solution we get must be primitive 
    return find_element_defining_embedding(O, d, t, all_slns, filter_func=check_sln)

## Experiments

Now we modify this to output some statistics. It outputs whether or not any embedding was found, and the number of $k$'s tried which give solutions, up until one is primitive.

In [5]:
from algorithm_5_1 import find_element_defining_embedding

def find_element_defining_primitive_embedding_with_statistics(O, d, t):
    global found_any_sln, ks
    """
    Finds element of order 'O' of trace t and norm d that is a primitive solution.
        Also returns whether any solution was found (primitive or imprimitive), and how many iterations of 'k' (which gave solutions) were required until we got a primitive solution
    """
    ks = []
    found_any_sln = False
    to_f_matrix = basis_to_matrix(O.basis()).inverse().transpose()

    def check_sln(alpha, k):
        global found_any_sln, ks
        found_any_sln = True
        if k not in ks:
            ks.append(k)
        return check_primitive(alpha, O, to_f_matrix)
    
    # By the validity check above, we know the solution we get must be primitive 
    alpha = find_element_defining_embedding(O, d, t, all_slns=False, filter_func=check_sln)
    return alpha, found_any_sln, len(ks) if alpha != None else None

The aim is to fix an quaternion order, fix the conductor of the quadratic order, and for randomly sampled $D$ (within a given range) see how long it takes to find a primitive embedding of the unique quadratic order in $\mathbb{Q}(\sqrt{-D})$ with that conductor. We will assume an embedding exists, and if it doesn't, resample $D$ and try again. We also vary the quaternion order to see how this changes, vary the conductor of the orders we are embedding, and try 2 different primes.

We start with a way of sampling generators of quadratic order of norm and conductor a certain size.

In [6]:
from hnf import upper_hnf_basis

def sample_order_with_conductor(min_D, max_D, C):
    """
    Samples a norm d of a trace zero element that generates a conductor C order (for random Q(sqrt{D}))
    """
    N = randrange(min_D, max_D)
    K.<z> = QuadraticField(-N)
    maximal_basis = upper_hnf_basis(K, K.maximal_order().basis()) # reduced so first element is 1
    generator = C * maximal_basis[1]
    return [generator.norm(), generator.trace()]

For our experiment we fix the following parameters:

In [7]:
# The range of D for sampling quadratic orders within Q(sqrt{D})
# we try:  2^10 - 2^12,   2^20 - 2^22,  and 2^30 - 2^32
min_D = 2^10
max_D = 2^12

# Conductor of orders to embed
# we try: 2, 5, 23, 4321, 2^30 + 3 (prime), 2^30 + 4540381 (smooth)
C = 2

# Prime
# we try: 419, or 2^15 + 63,   both are 3 mod 4
p = 419
B.<i,j,k> = QuaternionAlgebra(-1, -p)

# Quaternion order we are trying to embed in to
O = B.maximal_order()
# Or for p = 419
# O = B.quaternion_order([ 1/2 + 1/2*j, 1/6*i + 13/6*k, j, 3*k ])
# Or for p = 2^15 + 63
# O = B.quaternion_order([ 1/2 + 1/2*j, 1/6*i + 17/6*k, 3*j, k ])

# Number of successful embedding attempts until we stop
num_tries = 1000

And re-run the code below for different parameter choices:

In [9]:
results = []
while len(results) < num_tries:
    d, t = sample_order_with_conductor(min_D, max_D, C)
    _, foundSln, numtries = find_element_defining_primitive_embedding_with_statistics(O, d, t)
    if foundSln == False:
        continue
    results.append(numtries)
print("First is primtive: " + str(float(len([r for r in results if r == 1]) / len(results))) + "%")
print("No solutions primitive: " + str(float(len([r for r in results if r == None]) / len(results))) + "%")
print("First not-primitive, but later are: " + str(float(len([r for r in results if r > 1]) / len(results))) + "%")
for m in range(2, 10):
    print("  sln " + str(m) + " primitive: " + str(float(len([r for r in results if r == m]) / len(results))) + "%")
print("  slns > 9 primitive: " + str(float(len([r for r in results if r > 9]) / len(results))) + "%")

First is primtive: 0.814%
No solutions primitive: 0.182%
First not-primitive, but later are: 0.004%
  sln 2 primitive: 0.004%
  sln 3 primitive: 0.0%
  sln 4 primitive: 0.0%
  sln 5 primitive: 0.0%
  sln 6 primitive: 0.0%
  sln 7 primitive: 0.0%
  sln 8 primitive: 0.0%
  sln 9 primitive: 0.0%
  slns > 9 primitive: 0.0%


The results are summarized in the tables below, from which we made the following observations:

Observations:

- For $\mathcal{O}_0$, the majority of the time, we find a primitive solution with just 1 iteration. This is good news suggesting out algorithm is efficient at finding primitive solutions for $j=1728$. In most cases it's also high for other orders.
- Larger $D$ is better. Finding primitive solutions is faster, and there is a greater chance primitive solutions exist.
- The more factors the conductor has, the less likely the 1st solution we find will be primitive, but the probability a primitive solution is found within the first 5 iterations only decreases a little.
- Generally, the larger the conductor of the quadratic order the better - as long as there aren't lots more factors, and excluding the $C=1$ case where all solutions are primitive.
- For large $p$ it is worse for embedding a small norm $d$ ($d = C^2 D$ or $d = C^2 (N+1)/4$), but when you get up to large enough $d$ it's almost the same as for small p. For instance $C = 2$, with $D > p^2$ the results are very close to small $p$ results. This is expected as there are very few elements in an order of small norm, except for squares, so for small $D$ we are likely sampling more squares.

We start with results for $p = 419$, the tables on the left are for embedding into the maximal quaternion order $\mathcal{O}_0$, and those on the right are for quaternion order
$$\mathcal{O} = \left\langle \frac{1+j}{2}, \frac{i + 13 k}{6}, j, 3k \right\rangle_{\mathbb{Z}}$$
a short step from $\mathcal{O}_0$. We vary the size of $D$ in each, and try different conductors $C$ of the quadratic order.

**Conductor $C = 2$**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 84.25% | 90.25% | 89.25% | First solution primitive | 70.0% | 73.25% | 75.75% | 
| No solution primitive | 15.75% | 1.25% | 0.0% | No solution primitive | 22.0% | 0.25% | 0.0% | 
| &geq; 2nd solution primitive | 0.0% | 8.5% | 10.75% | &geq; 2nd solution primitive | 8.0% | 26.5% | 24.25% | 
| 2nd solution primitive | 0.0% | 8.25% | 10.25% | 2nd solution primitive | 7.75% | 20.25% | 19.25% | 
| 3rd solution primitive | 0.0% | 0.25% | 0.5% | 3rd solution primitive | 0.25% | 5.0% | 4.0% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.75% | 1.0% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.5% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Conductor $C = 5$**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 95.5% | 100.0% | 100.0% | First solution primitive | 89.0% | 92.0% | 93.25% | 
| No solution primitive | 4.5% | 0.0% | 0.0% | No solution primitive | 5.0% | 0.0% | 0.0% | 
| &geq; 2nd solution primitive | 0.0% | 0.0% | 0.0% | &geq; 2nd solution primitive | 6.0% | 8.0% | 6.75% | 
| 2nd solution primitive | 0.0% | 0.0% | 0.0% | 2nd solution primitive | 6.0% | 7.5% | 6.75% | 
| 3rd solution primitive | 0.0% | 0.0% | 0.0% | 3rd solution primitive | 0.0% | 0.5% | 0.0% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.0% | 0.0% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.0% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Conductor $C = 23$**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 90.25% | 93.75% | 95.0% | First solution primitive | 93.0% | 94.75% | 94.5% | 
| No solution primitive | 3.75% | 0.25% | 0.0% | No solution primitive | 1.0% | 0.0% | 0.0% | 
| &geq; 2nd solution primitive | 6.0% | 6.0% | 5.0% | &geq; 2nd solution primitive | 6.0% | 5.25% | 5.5% | 
| 2nd solution primitive | 6.0% | 5.5% | 4.25% | 2nd solution primitive | 6.0% | 5.0% | 5.25% | 
| 3rd solution primitive | 0.0% | 0.5% | 0.75% | 3rd solution primitive | 0.0% | 0.25% | 0.25% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.0% | 0.0% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.0% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Conductor $C = 4321$**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 98.75% | 100.0% | 100.0% | First solution primitive | 98.75% | 99.0% | 98.0% | 
| No solution primitive | 0.0% | 0.0% | 0.0% | No solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 2nd solution primitive | 1.25% | 0.0% | 0.0% | &geq; 2nd solution primitive | 1.25% | 1.0% | 2.0% | 
| 2nd solution primitive | 1.25% | 0.0% | 0.0% | 2nd solution primitive | 1.25% | 1.0% | 2.0% | 
| 3rd solution primitive | 0.0% | 0.0% | 0.0% | 3rd solution primitive | 0.0% | 0.0% | 0.0% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.0% | 0.0% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.0% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Condutor $C = 2^{30} + 3$ (a large prime)**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 98.25% | 100.0% | 100.0% | First solution primitive | 100.0% | 100.0% | 100.0% | 
| No solution primitive | 0.0% | 0.0% | 0.0% | No solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 2nd solution primitive | 1.75% | 0.0% | 0.0% | &geq; 2nd solution primitive | 0.0% | 0.0% | 0.0% | 
| 2nd solution primitive | 1.75% | 0.0% | 0.0% | 2nd solution primitive | 0.0% | 0.0% | 0.0% | 
| 3rd solution primitive | 0.0% | 0.0% | 0.0% | 3rd solution primitive | 0.0% | 0.0% | 0.0% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.0% | 0.0% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.0% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Conductor $C = 2^{30} + 4540381$ (a large smooth number)**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 70.5% | 69.75% | 64.5% | First solution primitive | 65.5% | 66.5% | 62.25% | 
| No solution primitive | 0.0% | 0.0% | 0.0% | No solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 2nd solution primitive | 29.5% | 30.25% | 35.5% | &geq; 2nd solution primitive | 34.5% | 33.5% | 37.75% | 
| 2nd solution primitive | 18.5% | 22.25% | 25.75% | 2nd solution primitive | 23.75% | 20.5% | 25.5% | 
| 3rd solution primitive | 9.25% | 6.25% | 7.75% | 3rd solution primitive | 7.5% | 8.5% | 8.5% | 
| 4th solution primitive | 1.75% | 1.25% | 1.5% | 4th solution primitive | 1.5% | 4.0% | 2.5% | 
| 5th solution primitive | 0.0% | 0.5% | 0.25% | 5th solution primitive | 1.5% | 0.25% | 0.75% | 
| 6th solution primitive | 0.0% | 0.0% | 0.25% | 6th solution primitive | 0.25% | 0.25% | 0.5% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Larger prime $p = 2^{15} + 63$**

**Conductor $C = 2$**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 28.75% | 86.25% | 91.5% | First solution primitive | 71.25% | 71.75% | 75.75% | 
| No solution primitive | 71.25% | 13.75% | 1.0% | No solution primitive | 28.75% | 26.25% | 2.5% | 
| &geq; 2nd solution primitive | 0.0% | 0.0% | 7.5% | &geq; 2nd solution primitive | 0.0% | 2.0% | 21.75% | 
| 2nd solution primitive | 0.0% | 0.0% | 7.5% | 2nd solution primitive | 0.0% | 2.0% | 17.25% | 
| 3rd solution primitive | 0.0% | 0.0% | 0.0% | 3rd solution primitive | 0.0% | 0.0% | 4.0% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.0% | 0.5% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.0% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Conductor $C = 23$**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 71.25% | 94.25% | 92.75% | First solution primitive | 91.5% | 93.5% | 92.5% | 
| No solution primitive | 28.75% | 3.5% | 0.5% | No solution primitive | 8.5% | 2.25% | 0.0% | 
| &geq; 2nd solution primitive | 0.0% | 2.25% | 6.75% | &geq; 2nd solution primitive | 0.0% | 4.25% | 7.5% | 
| 2nd solution primitive | 0.0% | 2.25% | 6.75% | 2nd solution primitive | 0.0% | 4.25% | 7.25% | 
| 3rd solution primitive | 0.0% | 0.0% | 0.0% | 3rd solution primitive | 0.0% | 0.0% | 0.25% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.0% | 0.0% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.0% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

**Conductor $C = 4321$**  
  
| Order $\mathcal{O}_0$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{10} \leq D \lt 2^{12}$ | $2^{20} \leq D \lt 2^{22}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 98.75% | 100.0% | 100.0% | First solution primitive | 98.5% | 97.75% | 98.0% | 
| No solution primitive | 1.25% | 0.0% | 0.0% | No solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 2nd solution primitive | 0.0% | 0.0% | 0.0% | &geq; 2nd solution primitive | 1.5% | 2.25% | 2.0% | 
| 2nd solution primitive | 0.0% | 0.0% | 0.0% | 2nd solution primitive | 1.5% | 2.25% | 1.75% | 
| 3rd solution primitive | 0.0% | 0.0% | 0.0% | 3rd solution primitive | 0.0% | 0.0% | 0.25% | 
| 4th solution primitive | 0.0% | 0.0% | 0.0% | 4th solution primitive | 0.0% | 0.0% | 0.0% | 
| 5th solution primitive | 0.0% | 0.0% | 0.0% | 5th solution primitive | 0.0% | 0.0% | 0.0% | 
| 6th solution primitive | 0.0% | 0.0% | 0.0% | 6th solution primitive | 0.0% | 0.0% | 0.0% | 
| 7th solution primitive | 0.0% | 0.0% | 0.0% | 7th solution primitive | 0.0% | 0.0% | 0.0% | 
| 8th solution primitive | 0.0% | 0.0% | 0.0% | 8th solution primitive | 0.0% | 0.0% | 0.0% | 
| 9th solution primitive | 0.0% | 0.0% | 0.0% | 9th solution primitive | 0.0% | 0.0% | 0.0% | 
| &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | &geq; 10th solution primitive | 0.0% | 0.0% | 0.0% | 

Finally we try one order further from $\mathcal{O}_0$ where the smallest connecting ideal has norm $114$:
$$\mathcal{O}_{furthest} = \left\langle \frac{1 + 35k}{2}, \frac{i + 415j + 2202k}{210}, 3j + 30k, 35k\right\rangle_{\mathbb{Z}}$$

**Conductor $C = 2$**  
  
| Order $\mathcal{O}_0$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}_{furthest}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 85.5% | First solution primitive | 74.75% | First solution primitive | 68.0% | 
| No solution primitive | 2.5% | No solution primitive | 4.0% | No solution primitive | 1.5% | 
| &geq; 2nd solution primitive | 12.0% | &geq; 2nd solution primitive | 21.25% | &geq; 2nd solution primitive | 30.5% | 
| 2nd solution primitive | 11.5% | 2nd solution primitive | 17.75% | 2nd solution primitive | 20.25% | 
| 3rd solution primitive | 0.5% | 3rd solution primitive | 2.75% | 3rd solution primitive | 8.25% | 
| 4th solution primitive | 0.0% | 4th solution primitive | 0.0% | 4th solution primitive | 0.75% | 
| 5th solution primitive | 0.0% | 5th solution primitive | 0.5% | 5th solution primitive | 0.5% | 
| 6th solution primitive | 0.0% | 6th solution primitive | 0.25% | 6th solution primitive | 0.75% | 
| 7th solution primitive | 0.0% | 7th solution primitive | 0.0% | 7th solution primitive | 0.0% | 
| 8th solution primitive | 0.0% | 8th solution primitive | 0.0% | 8th solution primitive | 0.0% | 
| 9th solution primitive | 0.0% | 9th solution primitive | 0.0% | 9th solution primitive | 0.0% | 
| &geq; 10th solution primitive | 0.0% | &geq; 10th solution primitive | 0.0% | &geq; 10th solution primitive | 0.0% | 

**Conductor $C = 4321$**  
  
| Order $\mathcal{O}_0$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}_{furthest}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 100.0% | First solution primitive | 98.25% | First solution primitive | 96.25% | 
| No solution primitive | 0.0% | No solution primitive | 0.0% | No solution primitive | 0.0% | 
| &geq; 2nd solution primitive | 0.0% | &geq; 2nd solution primitive | 1.75% | &geq; 2nd solution primitive | 3.75% | 
| 2nd solution primitive | 0.0% | 2nd solution primitive | 1.75% | 2nd solution primitive | 3.75% | 
| 3rd solution primitive | 0.0% | 3rd solution primitive | 0.0% | 3rd solution primitive | 0.0% | 
| 4th solution primitive | 0.0% | 4th solution primitive | 0.0% | 4th solution primitive | 0.0% | 
| 5th solution primitive | 0.0% | 5th solution primitive | 0.0% | 5th solution primitive | 0.0% | 
| 6th solution primitive | 0.0% | 6th solution primitive | 0.0% | 6th solution primitive | 0.0% | 
| 7th solution primitive | 0.0% | 7th solution primitive | 0.0% | 7th solution primitive | 0.0% | 
| 8th solution primitive | 0.0% | 8th solution primitive | 0.0% | 8th solution primitive | 0.0% | 
| 9th solution primitive | 0.0% | 9th solution primitive | 0.0% | 9th solution primitive | 0.0% | 
| &geq; 10th solution primitive | 0.0% | &geq; 10th solution primitive | 0.0% | &geq; 10th solution primitive | 0.0% | 

**Conductor $C = 2^{30} + 4540381$ (a large smooth number)**  
  
| Order $\mathcal{O}_0$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}$ | $2^{30} \leq D \lt 2^{32}$ | Order $\mathcal{O}_{furthest}$ | $2^{30} \leq D \lt 2^{32}$ | 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | 
| First solution primitive | 72.75% | First solution primitive | 65.5% | First solution primitive | 54.5% | 
| No solution primitive | 0.0% | No solution primitive | 0.0% | No solution primitive | 0.0% | 
| &geq; 2nd solution primitive | 27.25% | &geq; 2nd solution primitive | 34.5% | &geq; 2nd solution primitive | 45.5% | 
| 2nd solution primitive | 20.5% | 2nd solution primitive | 21.25% | 2nd solution primitive | 25.0% | 
| 3rd solution primitive | 4.75% | 3rd solution primitive | 9.5% | 3rd solution primitive | 11.25% | 
| 4th solution primitive | 1.25% | 4th solution primitive | 2.25% | 4th solution primitive | 6.0% | 
| 5th solution primitive | 0.5% | 5th solution primitive | 1.25% | 5th solution primitive | 1.25% | 
| 6th solution primitive | 0.25% | 6th solution primitive | 0.25% | 6th solution primitive | 1.0% | 
| 7th solution primitive | 0.0% | 7th solution primitive | 0.0% | 7th solution primitive | 0.5% | 
| 8th solution primitive | 0.0% | 8th solution primitive | 0.0% | 8th solution primitive | 0.0% | 
| 9th solution primitive | 0.0% | 9th solution primitive | 0.0% | 9th solution primitive | 0.0% | 
| &geq; 10th solution primitive | 0.0% | &geq; 10th solution primitive | 0.0% | &geq; 10th solution primitive | 0.5% | 